In [21]:
import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import numpy as np
from sqlalchemy import Numeric, Date, create_engine, select, insert
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
import matplotlib.pyplot as plt
from db_tables import *
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()
url = os.getenv('PSQL_URL')

### Method 1: Manual Way

In [9]:
engine = create_engine(url, echo=True)
stmt = select(GoldRates)
rows = []
with Session(engine) as session:
    for i, row in enumerate(session.execute(stmt)):
        rows.append(row)

2023-11-13 16:40:11,687 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-13 16:40:11,688 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 16:40:12,214 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-13 16:40:12,215 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 16:40:13,115 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-13 16:40:13,117 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-13 16:40:13,573 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 16:40:13,576 INFO sqlalchemy.engine.Engine SELECT goldrates.pricedate, goldrates.price22k, goldrates.price24k, goldrates.price18k 
FROM goldrates
2023-11-13 16:40:13,577 INFO sqlalchemy.engine.Engine [generated in 0.00097s] {}
2023-11-13 16:40:14,535 INFO sqlalchemy.engine.Engine ROLLBACK


In [14]:
rows[0][0]

GoldRates(pricedate=2021-07-01, price22k=202.0, price24k=215.0, price18k=165.25)

In [13]:
df_data = [[row[0].pricedate, row[0].price22k, row[0].price24k, row[0].price18k] for row in rows]

In [16]:
df = pd.DataFrame(df_data, columns=['Date', '22k', '24k', '18k'])

In [18]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 865 entries, 2021-07-01 to 2023-11-13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   22k     865 non-null    object
 1   24k     865 non-null    object
 2   18k     865 non-null    object
dtypes: object(3)
memory usage: 27.0+ KB


### Automated

In [26]:
df_2 = pd.read_sql(select(GoldRates), engine, parse_dates=['pricedate'], index_col='pricedate')

2023-11-13 16:47:55,692 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 16:47:55,693 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-11-13 16:47:55,694 INFO sqlalchemy.engine.Engine [cached since 25.95s ago] {'table_name': <sqlalchemy.sql.selectable.Select object at 0x7fe7c1a8eee0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-11-13 16:47:55,695 INFO sqlalchemy.engine.Engine SELECT goldrates.pricedate, goldrates.price22k, goldrates.price24k, goldrates.price18k 
FROM goldrates
2023-11-13 16:47:55,696

In [28]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 865 entries, 2021-07-01 to 2023-11-13
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price22k  865 non-null    float64
 1   price24k  865 non-null    float64
 2   price18k  865 non-null    float64
dtypes: float64(3)
memory usage: 27.0 KB
